# Integrating an Observation Tool in an Agent - SOLUTION

> **Learning Outcomes:**
> - Create an observation tool with AutoGen.
> - Integrate the observation tool with an agent.
> - Understand the AutoGen Agent flow.

## Introduction

In this lab, we will implement an observation tool that will allow the agent to observe their environment. This tool won't have any affect on the world outside of the agent, but will allow the agent to gather information about that world. This is a useful capacity for an agent to have, as it allows the agent to make more informed decisions.

### AutoGen Framework
[AutoGen](https://microsoft.github.io/autogen/stable/index.html) is "a framework for building AI agents and applications" that is developed by Microsoft. It is designed to be a flexible and extensible framework that can be used to build a wide range of AI applications.

We will be working with the `autogen-agentchat` component which is designed for building conversational single and multi-agent systems. This component is built on top of the `autogen` framework and provides a simple way to build agents that can communicate with each other.


### The Scenario
Our agent will be tasked with converting Wikipedia pages into a timeline. The agent will be able to observe the Wikipedia page and extract the information needed to create the timeline. The agent will then be able to use this information to create a timeline of the events described in the Wikipedia page. If the agent needs additional information, it can perform more observations to gather that information.


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [1]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU \
    openai==1.95.* \
    autogen-agentchat==0.6.* \
    autogen-ext[openai]==0.6.* \
    nlpia2-wikipedia==1.5.*

import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
langchain-openai 0.3.35 requires openai<3.0.0,>=1.104.2, but you have openai 1.95.1 which is incompatible.
pyppeteer 2.0.0 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.5.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.
spyder 5.2.2 requires pyqt5<5.13, but you have pyqt5 5.15.11 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Enter your OpenAI API key:  ········


## Core

### Step 1 - Write Query Function

We will use the `nlpia2-wikipedia` library to work with Wikipedia. This library provides three functions, which are demonstrated in the cells below.

In [3]:
import wikipedia

results = wikipedia.search("New York City") # List of titles
results

['New York City',
 'Mayor of New York City',
 'List of mayors of New York City',
 'Boroughs of New York City',
 'New York City mayoral elections',
 'Demographics of New York City',
 'New York',
 'New York (state)',
 '2013 New York City mayoral election',
 'Economy of New York City']

In [4]:
# Summary of the first title
summary = wikipedia.summary(results[0])
summary

{'summary': "New York, often called New York City (NYC), is the most populous city in the United States. It is located at the southern tip of New York State on New York Harbor, one of the world's largest natural harbors. The city comprises five boroughs, each coextensive with its respective county. It is the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the United States by both population and urban area. New York is a global center of finance and commerce, culture, technology, entertainment and media, academics and scientific output, the arts and fashion, and, as home to the headquarters of the United Nations, international diplomacy.\nWith an estimated population in July 2024 of 8,478,072, distributed over 300.46 square miles (778.2 km2), the city is the most densely populated major city in the United States. New York City has more than double the population of Los Angeles, the country's seco

In [5]:
#Full page of the first title
page = wikipedia.page(results[0])
print('Title:', page.title)
print('URL:', page.url)
print('Content:', page.content[:1000])
print('Summary:', page.summary[:1000])
print('Links:', page.links)

Title: New York City
URL: https://en.wikipedia.org/wiki/New_York_City
Content: New York, often called New York City (NYC), is the most populous city in the United States. It is located at the southern tip of New York State on New York Harbor, one of the world's largest natural harbors. The city comprises five boroughs, each coextensive with its respective county. It is the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the United States by both population and urban area. New York is a global center of finance and commerce, culture, technology, entertainment and media, academics and scientific output, the arts and fashion, and, as home to the headquarters of the United Nations, international diplomacy.
With an estimated population in July 2024 of 8,478,072, distributed over 300.46 square miles (778.2 km2), the city is the most densely populated major city in the United States. New York City has m

Most Agentic libraries, including AutoGen, allow tools to be written as functions. These functions are then converted to tools by the library for easy integration with the agent. While the exact method of writing these functions may vary depending on the library, the general process is the same.

1. Write a function that takes the necessary inputs and returns the desired output.
2. Add type hints to the function to specify the types of the inputs and outputs.
3. Create a description of the function.


```python
def hello_world(name: str) -> str:
    return f"Hello, {name}!"
```

In the cell below, write a function called `get_wikipedia_page` that takes a string as input and returns the content of the Wikipedia page for that string.

In [6]:
# YOUR CODE HERE
# [SOLUTION]
def get_wikipedia_page(title: str) -> str:
    return wikipedia.page(title).content


A function on it's own is not a tool. It needs to be converted to a tool using the `autogen` library. This is done by using the `FunctionTool` class. This class takes the function and a description of the function as input and creates a tool that can be used by the agent.

```python
tool = FunctionTool(hello_world, description="Greets the person by name.")
await tool.run_json({"title": "John"})
```

In the cell below, create a `FunctionTool` for the your function and test it by running the tool directly. Save the tool to the variable `page_tool`.

In [7]:
from autogen_core.tools import FunctionTool
from autogen_core import CancellationToken

# YOUR CODE HERE
#[SOLUTION]
page_tool = FunctionTool(get_wikipedia_page, description="Content of a Wikipedia page given its title")

await page_tool.run_json({"title": "Carillon"}, CancellationToken())

'A carillon (US:  KARR-ə-lon, UK:  kə-RIL-yən) is a pitched percussion instrument that is played with a keyboard and consists of at least 23 bells. The bells are cast in bronze, hung in fixed suspension, and tuned in chromatic order so that they can be sounded harmoniously together. They are struck with clappers connected to a keyboard of wooden batons played with the hands and pedals played with the feet. Often housed in bell towers, carillons are usually owned by churches, universities, or municipalities. They can include an automatic system through which the time is announced and simple tunes are played throughout the day.\nCarillons come in many designs, weights, sizes, and sounds. They are among the world\'s heaviest instruments, and the heaviest carillon weighs over 91 metric tons (100 short tons). Most weigh between 4.5 and 15 metric tons (5.0 and 16.5 short tons). To be considered a carillon, a minimum of 23 bells is needed; otherwise, it is called a chime. Standard-sized instr

### Step 2 - Creating an Agent

Now that you have a tool, we need to integrate it with the agent. AutoGen's `AssistantAgent` class has the built-in capability to use a tool and respond based on it's output. However, it can only use one tool, once per task. To allow the the agent to use the tool multiple times, we can place it in a round-robin group chat by itself. This will allow the agent to continue working until it is complete.

Run the cell below to see how the agent uses the tool.

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
import textwrap

model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")
agent = AssistantAgent(
    name="timeline_creator",
    model_client=model_client,
    tools=[page_tool],
    system_message=textwrap.dedent("""
        You are an AI assistant that can help users create timelines of events based on the content of Wikipedia pages.
        When the user requests a timeline, perform the following tasks:

        1. Query Wikipedia for the content of the page.
        2. Use the content to extract dates.
        3. Use the extracted dates for form a timeline.

        Reply with TERMINATE when the task has been completed.
    """.strip()),
)

team = RoundRobinGroupChat([agent], termination_condition=TextMentionTermination("TERMINATE"))


await Console(team.run_stream(task="Create a timeline of Python."))

---------- TextMessage (user) ----------
Create a timeline of Python.
---------- ToolCallRequestEvent (timeline_creator) ----------
[FunctionCall(id='call_71YUnjtnN0e1DteMxg8G2mVf', arguments='{"title":"Python (programming language)"}', name='get_wikipedia_page')]


query: {'normalized': [{'from': 'Python_(Programming_Language)', 'to': 'Python (Programming Language)'}], 'redirects': [{'from': 'Python (Programming Language)', 'to': 'Python (programming language)'}], 'pages': {'23862': {'pageid': 23862, 'ns': 0, 'title': 'Python (programming language)', 'contentmodel': 'wikitext', 'pagelanguage': 'en', 'pagelanguagehtmlcode': 'en', 'pagelanguagedir': 'ltr', 'touched': '2025-11-23T04:31:37Z', 'lastrevid': 1323413324, 'length': 138612, 'fullurl': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'editurl': 'https://en.wikipedia.org/w/index.php?title=Python_(programming_language)&action=edit', 'canonicalurl': 'https://en.wikipedia.org/wiki/Python_(programming_language)'}}}


---------- ToolCallExecutionEvent (timeline_creator) ----------
[FunctionExecutionResult(content='Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Beginning with Python 3.5, capabilities and keywords for typing were added to the language, allowing optional static typing. Currently only versions in the 3.x series are supported. \nPython has gained widespread use in the machine learning community. It is widely taught as an introductory programming language. Since 2003, Python ha

TaskResult(messages=[TextMessage(id='21140357-bcaf-409c-bb50-2224fa17d398', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 9, 223126, tzinfo=datetime.timezone.utc), content='Create a timeline of Python.', type='TextMessage'), ToolCallRequestEvent(id='3616a8c7-70fc-4d4d-9d61-0c0d709bb6ef', source='timeline_creator', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 10, 99831, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_71YUnjtnN0e1DteMxg8G2mVf', arguments='{"title":"Python (programming language)"}', name='get_wikipedia_page')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='7c92d27e-62b7-43bc-bf41-dfe418e01ab1', source='timeline_creator', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 11, 157296, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Python is a high-level, gene

### Step 3 - Extending the Capabilities

Now that we have a tool that can get the content of a Wikipedia page, we can extend the agent's capabilities by adding more tools. The Wikipedia library can also search for pages based on a query.

In the cell below, write a function called `search_wikipedia` that takes a string as input and returns the list of Wikipedia pages that match that string. Use that function to create a tool called `search_tool` and test it by running the tool directly.

In [9]:
# YOUR CODE HERE
# [SOLUTION]
def search_wikipedia(query: str) -> list:
    return wikipedia.search(query)

search_tool = FunctionTool(search_wikipedia, description="Search Wikipedia for a given query")
await search_tool.run_json({"query": "Python"}, CancellationToken())

['Python (programming language)',
 'Python',
 'Monty Python',
 'Python (codename)',
 'Reticulated python',
 'Ball python',
 'History of Python',
 'Python (missile)',
 'Burmese python',
 'Indian python']

Now that we have a tool that can search for Wikipedia pages, we can integrate it with the agent. In the cell below, create a new `AssistantAgent` and round-robin group chat that uses the `search_tool` and `page_tool` tools. Test the agent by running the cell.

In [10]:
# YOUR CODE HERE
# [SOLUTION]
model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")
agent = AssistantAgent(
    name="timeline_creator",
    model_client=model_client,
    tools=[page_tool, search_tool],
    system_message=textwrap.dedent("""
        You are an AI assistant that can help users create timelines of events based on the content of Wikipedia pages.
        When the user requests a timeline, perform the following tasks:

        1. Search wikipedia for the correct page.
        2. Query Wikipedia for the content of the page.
        3. Use the content to extract dates.
        4. Use the extracted dates for form a timeline.

        Reply with TERMINATE when the task has been completed.
    """.strip()),
)

team = RoundRobinGroupChat([agent], termination_condition=TextMentionTermination("TERMINATE"))


await Console(team.run_stream(task="Create a timeline of the musical instrument that is a tall tower with bells played from a keyboard."))

---------- TextMessage (user) ----------
Create a timeline of the musical instrument that is a tall tower with bells played from a keyboard.
---------- ToolCallRequestEvent (timeline_creator) ----------
[FunctionCall(id='call_MTZ2F0wwDvUhF10W52u7zOEr', arguments='{"query":"carillon"}', name='search_wikipedia')]
---------- ToolCallExecutionEvent (timeline_creator) ----------
[FunctionExecutionResult(content="['Carillon', 'Carillon (disambiguation)', 'List of carillons', 'List of carillons in the United States', 'Battle of Carillon', 'Carillon, Quebec', 'List of carillons in Belgium', 'List of carillons in Germany', 'Netherlands Carillon', 'Electronic carillon']", name='search_wikipedia', call_id='call_MTZ2F0wwDvUhF10W52u7zOEr', is_error=False)]
---------- ToolCallSummaryMessage (timeline_creator) ----------
['Carillon', 'Carillon (disambiguation)', 'List of carillons', 'List of carillons in the United States', 'Battle of Carillon', 'Carillon, Quebec', 'List of carillons in Belgium', 'Li

TaskResult(messages=[TextMessage(id='de33f341-61a3-4504-96bb-06a00bf76444', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 14, 215576, tzinfo=datetime.timezone.utc), content='Create a timeline of the musical instrument that is a tall tower with bells played from a keyboard.', type='TextMessage'), ToolCallRequestEvent(id='26b11886-b4a8-4344-91f5-cff032256899', source='timeline_creator', models_usage=RequestUsage(prompt_tokens=196, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 14, 897344, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_MTZ2F0wwDvUhF10W52u7zOEr', arguments='{"query":"carillon"}', name='search_wikipedia')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='ee802a7e-6007-4f3d-86b4-20258ba34005', source='timeline_creator', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 0, 15, 190791, tzinfo=datetime.timezone.utc), content=[FunctionExe

Great Job!

Now you have an agent that can search for and read Wikipedia pages. You can extend the agent's capabilities by adding more tools to it. In the challenges below, we'll do just that.

## Bonus Challenge 1 - File System Agent

Create a set of tools that allows an agent to search for and read files on the local file system. Integrate these tools with the agent and test them.

In [11]:
# YOUR CODE HERE

## Bonus Challenge 2 - API Access Tools

A common need is for an agent to access an API to gather information. [public-apis](https://github.com/public-apis/public-apis) is a list of public APIs that can be easily integrated into AI Agents. Select an API from the list and create a set of tools that allow an agent to access that API. Integrate these tools with the agent and test them.

If you need a suggestion, try the [US National Weather Service API](https://www.weather.gov/documentation/services-web-api).

In [12]:
# YOUR CODE HERE

# Conclusion

In this lab, we created an observation tool that allows an agent to gather information about the world around it. We integrated this tool with an agent and used it to create a timeline of events from a Wikipedia page. We then extended the agent's capabilities by adding more tools to it.

In the challenges, we created a set of tools that allow an agent to search for and read files on the local file system and access an API to gather information. We integrated these tools with the agent and tested them.

Integrating tools with agents is essential for building AI applications that can interact with the world around them. By creating tools that allow agents to observe and interact with their environment, we can build agents that can perform a wide range of tasks.

